In [14]:
pip install gradio


In [19]:
#SAEESH KOLGE


import numpy as np
import pandas as pd
import yfinance as yf
import gradio as gr
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, r2_score

# Fetch stock data
def fetch_stock_data(ticker, start, end):
    data = yf.download(ticker, start=start, end=end)
    return data

# Prepare data
def prepare_data(data):
    data['Tomorrow'] = data['Close'].shift(-1)
    data.dropna(inplace=True)
    X = data[['Open', 'High', 'Low', 'Close', 'Volume']]
    y = data['Tomorrow']
    return X, y

# Model mapping
models = {
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Linear Regression': LinearRegression(),
    'Support Vector Regressor': SVR(),
}

def predict_stock(ticker, start_date, end_date, selected_model_name):
    data = fetch_stock_data(ticker, start_date, end_date)
    if data.empty:
        return 'No data found for the selected ticker and date range.'

    X, y = prepare_data(data)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = models[selected_model_name]
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    latest_data = X.iloc[-1].values.reshape(1, -1)
    predicted_price = model.predict(latest_data)[0]

    return f"Mean Absolute Error: {mae:.2f}\nR² Score: {r2:.2f}\nPredicted price for next day: {predicted_price:.2f}"

# Gradio UI
demo = gr.Interface(
    fn=predict_stock,
    inputs=[
        gr.Textbox(label="Enter Ticker Symbol", value="GC=F"),
        gr.Textbox(label="Start Date", value="2020-01-01"),
        gr.Textbox(label="End Date", value="2024-01-01"),
        gr.Radio(list(models.keys()), label="Select Model")
    ],
    outputs="text"
)

demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1d6d6a4289e2fd8f9c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
